In [120]:
import requests
import os
import time

In [113]:
client_id = "m75coxowpb3z2ebz6wmy2xgplfpyea"
client_secret = "e49c4j2r7murvlkk64ovkc7s5pyl2e"
access_token = "fsrvvbklii8qamk05i42jhyc9nx3ke"

api_url = "https://api.igdb.com/"

headers = {
    'Client-ID': client_id,
    'Authorization': f'Bearer {access_token}'
}

In [116]:
import pandas as pd
df = pd.read_csv("vgsales.csv")
df.head()

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37


In [ ]:
from rapidfuzz import fuzz
import time
from datetime import datetime

import os
OUTPUT_FILE = "enriched_games.csv"
SAVE_EVERY = 10

if not os.path.exists(OUTPUT_FILE):
    pd.DataFrame(columns=['Name', 'IGDB_Name', 'Aggregated_Rating', 'Age_Ratings_IDs']).to_csv(OUTPUT_FILE, index=False)

existing_df = pd.read_csv(OUTPUT_FILE)
already_enriched = set(existing_df['Name'].tolist())

def search_game_igdb(game_name):
    query = f'search "{game_name}"; fields name, first_release_date, aggregated_rating, age_ratings, rating;'
    response = requests.post('https://api.igdb.com/v4/games', headers=headers, data=query)
    if response.status_code == 200:
        return response.json()
    return []

def find_best_match(game_name, game_year, igdb_results):
    best_score = 0
    best_game_data = {
        'name': None,
        'rating': None,
        'rating_ids': []
    }
    for game in igdb_results:
        name = game.get('name', '')
        score = fuzz.ratio(name.lower(), game_name.lower())

        timestamp = game.get('first_release_date')
        igdb_year = None
        if timestamp:
            try:
                igdb_year = datetime.utcfromtimestamp(timestamp).year
            except:
                igdb_year = 0
            if abs(igdb_year - game_year) <= 2:
                score += 10  
            if abs(igdb_year - game_year) <= 1:
                score += 20
            if abs(igdb_year - game_year) == 0:
                score += 30

        if score > best_score and score >= 80:
            best_score = score
            best_game_data['name'] = game.get('name')
            best_game_data['rating'] = game.get('aggregated_rating') or game.get('rating')
            best_game_data['rating_ids'].extend(game.get('age_ratings') or [])

    best_game_data['rating_ids'] = list(set(best_game_data['rating_ids']))
    
    return best_game_data if best_score >= 85 else None

buffer = []
counter = 0

top_100 = df.sort_values(by='Rank').copy()

for _, row in top_100.iterrows():
    game_name = row['Name']
    if game_name in already_enriched:
        continue

    igdb_results = search_game_igdb(game_name)
    best_match = find_best_match(game_name, int(row['Year']) if not pd.isna(row['Year']) else 0, igdb_results)
    row_data = {
        'Name': game_name,
        'IGDB_Name': best_match.get('name') if best_match else None,
        'Aggregated_Rating': best_match.get('rating') if best_match else None,
        'Age_Ratings_IDs': best_match.get('rating_ids') if best_match else None
    }

    buffer.append(row_data)
    counter += 1

    if counter % SAVE_EVERY == 0:
        print("Row rank:,", row["Rank"])
        pd.DataFrame(buffer).to_csv(OUTPUT_FILE, mode='a', header=False, index=False)
        buffer.clear()

    time.sleep(0.1)

if buffer:
    pd.DataFrame(buffer).to_csv(OUTPUT_FILE, mode='a', header=False, index=False)

In [126]:
import ast

df = pd.read_csv(OUTPUT_FILE)
valid_entries = df['Age_Ratings_IDs'].dropna()
valid_entries = valid_entries[valid_entries != '[]']


all_ids = set()
for entry in valid_entries:
    try:
        ids = ast.literal_eval(entry)
        all_ids.update(ids)
    except (ValueError, SyntaxError):
        continue


print(len(all_ids))

def batch_query_age_ratings_buffered(all_ids, headers, batch_size=500, save_every=500, output_file="ratings.csv"):
    buffer = []
    counter = 0
    all_ids = list(all_ids)

    for i in range(0, len(all_ids), batch_size):
        batch = all_ids[i:i+batch_size]
        ids_string = ','.join(map(str, batch))
        query = f'fields id, rating, category; where id = ({ids_string}); limit {batch_size};'

        response = requests.post('https://api.igdb.com/v4/age_ratings', headers=headers, data=query)

        if response.status_code == 200:
            buffer.extend(response.json())
            counter += len(batch)

            if counter % save_every == 0:
                pd.DataFrame(buffer).to_csv(output_file, mode='a', header=not os.path.exists(output_file), index=False)
                print(f"Saved {counter} records to {output_file}")
                buffer.clear()

        else:
            print(f"Error {response.status_code}: {response.text}")

        time.sleep(0.25)

    if buffer:
        pd.DataFrame(buffer).to_csv(output_file, mode='a', header=not os.path.exists(output_file), index=False)
        print(f"Saved final {len(buffer)} records.")

batch_query_age_ratings_buffered(all_ids, headers)

18092
Saved 500 records to ratings.csv
Saved 1000 records to ratings.csv
Saved 1500 records to ratings.csv
Saved 2000 records to ratings.csv
Saved 2500 records to ratings.csv
Saved 3000 records to ratings.csv
Saved 3500 records to ratings.csv
Saved 4000 records to ratings.csv
Saved 4500 records to ratings.csv
Saved 5000 records to ratings.csv
Saved 5500 records to ratings.csv
Saved 6000 records to ratings.csv
Saved 6500 records to ratings.csv
Saved 7000 records to ratings.csv
Saved 7500 records to ratings.csv
Saved 8000 records to ratings.csv
Saved 8500 records to ratings.csv
Saved 9000 records to ratings.csv
Saved 9500 records to ratings.csv
Saved 10000 records to ratings.csv
Saved 10500 records to ratings.csv
Saved 11000 records to ratings.csv
Saved 11500 records to ratings.csv
Saved 12000 records to ratings.csv
Saved 12500 records to ratings.csv
Saved 13000 records to ratings.csv
Saved 13500 records to ratings.csv
Saved 14000 records to ratings.csv
Saved 14500 records to ratings.csv

In [127]:
ratings_df = pd.read_csv("ratings.csv")
print(ratings_df.head())

id_to_info = ratings_df.set_index('id')[['category', 'rating']].to_dict(orient='index')

pegi_enum = {
    1: "PEGI 3",
    2: "PEGI 7",
    3: "PEGI 12",
    4: "PEGI 16",
    5: "PEGI 18"
}

esrb_enum = {
    6: "RP",
    7: "EC",
    8: "E",
    9: "E10+",
    10: "T",
    11: "M",
    12: "AO"
}

def get_rating_labels(age_rating_ids_str):
    try:
        ids = ast.literal_eval(age_rating_ids_str)
    except:
        return []

    labels = []

    for id_ in ids:
        info = id_to_info.get(id_)
        if not info:
            continue

        category = info['category']
        rating = info['rating']

        if category == 2 and rating in pegi_enum:
            labels.append(pegi_enum[rating])
        elif category == 1 and rating in esrb_enum:
            labels.append(esrb_enum[rating])

    return labels

encriched_games_df = pd.read_csv("enriched_games.csv")

encriched_games_df['Age_Rating_Label'] = encriched_games_df['Age_Ratings_IDs'].apply(get_rating_labels)

encriched_games_df.to_csv("enriched_games_with_labels.csv")

      id  category  rating
0  65987         1      10
1  65808         1      10
2  65663         1      10
3  65193         1      10
4    210         1       8


In [128]:
original_df = pd.read_csv("vgsales.csv")
enriched_df = pd.read_csv("enriched_games_with_labels.csv")

columns_to_merge = ['Name', 'Aggregated_Rating', 'Age_Rating_Label']
merge_df = enriched_df[['Name', 'Aggregated_Rating', 'Age_Rating_Label']].drop_duplicates('Name')
merged_df = original_df.merge(merge_df, on='Name', how='left')

final_output_path = "original_plus_rating_labels.csv"
merged_df.to_csv(final_output_path, index=False)

In [ ]:

df = pd.read_csv("original_plus_rating_labels.csv")


missing_year_df = df[df['Year'].isna()]
print(len(missing_year_df))
missing_year_df.to_csv("missing_year.csv")
OUTPUT_FILE = "missing_year_filled.csv"

def search_game_igdb(game_name):
    query = f'search "{game_name}"; fields name, first_release_date, aggregated_rating, age_ratings, rating; limit 10;'
    response = requests.post('https://api.igdb.com/v4/games', headers=headers, data=query)
    if response.status_code == 200:
        return response.json()
    return []

def find_best_match(game_name, igdb_results):
    best_score = 0
    best_game_data = {
        'name': None,
        'year': None,
        'rating': None,
        'rating_ids': []
    }

    for game in igdb_results:
        name = game.get('name', '')
        score = fuzz.ratio(name.lower(), game_name.lower())

        timestamp = game.get('first_release_date')
        igdb_year = None
        if timestamp:
            print("HAVE TIMESTAMP")
            try:
                igdb_year = datetime.utcfromtimestamp(timestamp).year
            except:
                igdb_year = None

        if score >= 80:
            # Bonus for having a year at all
            if igdb_year:
                score += 10
            if score > best_score:
                best_score = score
                best_game_data['name'] = name
                best_game_data['year'] = igdb_year
                best_game_data['rating'] = game.get('aggregated_rating') or game.get('rating')
                best_game_data['rating_ids'] = list(set(game.get('age_ratings') or []))

    return best_game_data if best_score >= 85 else None

# Main enrichment loop
buffer = []
counter = 0

for _, row in missing_year_df.iterrows():
    game_name = row['Name']
    igdb_results = search_game_igdb(game_name)
    best_match = find_best_match(game_name, igdb_results)

    if best_match:
        row_data = {
            'Name': game_name,
            'IGDB_Name': best_match['name'],
            'Year': best_match['year'],
            'Aggregated_Rating': best_match['rating'],
            'Age_Ratings_IDs': best_match['rating_ids']
        }
        buffer.append(row_data)
        counter += 1

    if counter % SAVE_EVERY == 0 and buffer:
        pd.DataFrame(buffer).to_csv(OUTPUT_FILE, mode='a', header=not pd.io.common.file_exists(OUTPUT_FILE), index=False)
        print(f"Saved {counter} rows...")
        buffer.clear()

    time.sleep(0.25)  # be nice to the API

# Final save
if buffer:
    pd.DataFrame(buffer).to_csv(OUTPUT_FILE, mode='a', header=not pd.io.common.file_exists(OUTPUT_FILE), index=False)
    print(f"Saved final {len(buffer)} rows.")

In [ ]:
full_df = pd.read_csv("original_plus_rating_labels.csv")
filled_df = pd.read_csv("missing_year_filled.csv")

name_to_year = dict(zip(filled_df['Name'], filled_df['Year']))
def fill_year(row):
    if pd.isna(row['Year']):
        return name_to_year.get(row['Name'], row['Year'])
    return row['Year']


full_df['Year'] = full_df.apply(fill_year, axis=1)
full_df.to_csv("original_plus_rating_labels.csv", index=False)

In [ ]:
full_df_no_year = full_df[full_df['Year'].isna()]
full_df_no_year.to_csv('no_year.csv')

In [ ]:
import pandas as pd

full_df = pd.read_csv("original_plus_rating_labels.csv")

missing_publishers_df = full_df[full_df['Publisher'].isna()]
missing_publishers_df.to_csv("missing_publisher.csv")


publisher_counts = full_df['Publisher'].value_counts().reset_index()
publisher_counts.columns = ['Publisher', 'Count']
publisher_counts.to_csv("publisher_counts.csv")

In [ ]:
updated_missing_publishers_df = missing_publishers_df.copy()

# Keyword map per alguns que estan repetits / son varis de la mateixa categoria
# La resta imputació manual en base les dades de https://gamefaqs.gamespot.com/
keyword_to_publisher = {
    "nick": "Nintendo",
    "cartoon network": "Majesco Entertainment",
    "sonic": "Sega",
    "spongebob": "Majesco Entertainment",
    "fairly": "Majesco Entertainment",
    "zelda": "Nintendo",
    "dragon": "Atari",
    "game boy advance video volume": "Majesco Entertainment",
    "poke": "Nintendo",
    "shre": "Majesco Entertainment"
}

def assign_publisher_by_keyword(title):
    title_lower = str(title).lower()
    for keyword, publisher in keyword_to_publisher.items():
        if keyword in title_lower:
            return publisher
    return None

updated_missing_publishers_df['Publisher'] = updated_missing_publishers_df.apply(
    lambda row: assign_publisher_by_keyword(row['Name']) if pd.isna(row['Publisher']) else row['Publisher'],
    axis=1
)

sorted_missing_publishers_df = updated_missing_publishers_df.sort_values(by='Publisher', na_position='first')

# Save to CSV
output_path = 'missing_publishers.csv'
sorted_missing_publishers_df.to_csv(output_path, index=False)



In [ ]:
full_df = pd.read_csv("original_plus_rating_labels.csv")
filled_df = pd.read_csv("missing_publishers_filled.csv")

name_to_year = dict(zip(filled_df['Name'], filled_df['Publisher']))
def fill_year(row):
    if pd.isna(row['Publisher']):
        return name_to_year.get(row['Name'], row['Publisher'])
    return row['Year']


full_df['Publisher'] = full_df.apply(fill_year, axis=1)
full_df.to_csv("original_plus_rating_labels.csv", index=False)

In [ ]:
full_df.head()

In [ ]:
full_df = pd.read_csv("original_plus_rating_labels.csv")
import ast


full_df['Age_Rating_Label'] = full_df.apply(lambda row: list(set(ast.literal_eval(row['Age_Rating_Label']))), axis=1)
full_df.head(40)

full_df.to_csv("original_plus_rating_labels.csv", index=False)


In [ ]:
import numpy as np

label_to_age = {
    'E': 0,
    'PEGI 7': 7,
    'PEGI 3': 3,
    'PEGI 12': 12,
    'M': 17,
    'PEGI 18': 18,
    'PEGI 16': 16,
    'E10+': 10,
    'T': 13,
    'EC': 5,
    'RP': np.nan
}

def labels_to_rating(label_list):
    max_rating = np.nan
    for label in label_list:
        
        age = label_to_age.get(label, np.nan)  # Default to NaN if label is unknown
        if (pd.isna(max_rating) or (age > max_rating)):
            max_rating = age
    return max_rating

full_df['Age_Rating'] = full_df['Age_Rating_Label'].apply(labels_to_rating)

full_df.head(50)

full_df.to_csv("original_plus_rating_labels_and_ratings.csv", index=False)

In [ ]:

df = pd.read_csv("original_plus_rating_labels_and_ratings.csv")
df.info()

# Imputar ratings

df['Aggregated_Rating'] = df.groupby(['Genre', 'Platform'])['Aggregated_Rating'].transform(
    lambda x: x.fillna(x.median()))

df['Aggregated_Rating'] = df['Aggregated_Rating'].fillna(df['Aggregated_Rating'].median())

df['Age_Rating'] = df.groupby(['Genre', 'Platform'])['Age_Rating'].transform(
    lambda x: x.fillna(x.mode()[0] if len(x.mode()) > 0 else 0))

df.info()
df.to_csv("original_plus_rating_labels_and_ratings_imputed.csv", index=False)

In [ ]:

df = pd.read_csv("original_plus_rating_labels_and_ratings_imputed.csv")
display(df.head())


#Imputar labels
age_to_label = {value: key for key, value in label_to_age.items()}

def fill_age_rating_label(row):
    if len(ast.literal_eval(row['Age_Rating_Label'])) == 0:  # Check if the list is empty
        age = row['Age_Rating']
        if age in age_to_label:
            return [age_to_label[age]]
    return row['Age_Rating_Label']

df['Age_Rating_Label'] = df.apply(fill_age_rating_label, axis=1)

df.head()

df.to_csv("original_plus_rating_labels_and_ratings_imputed.csv", index=False)


In [ ]:
import openpyxl
final_df = pd.read_csv("original_plus_rating_labels_and_ratings_imputed.csv")
final_df['Aggregated_Rating'] = final_df['Aggregated_Rating'].round(1)
final_df.to_excel('final_dataset.xlsx', index=False)

In [ ]:

og_df = pd.read_csv("vgsales.csv")

df['Publisher'] = df['Publisher'].where(og_df['Publisher'].isna(), og_df['Publisher'])

display(df)

df['Publisher'].value_counts()

display(df[df['Publisher'].isna()])

df.to_csv("original_plus_rating_labels_and_ratings_imputed.csv", index=False)
df.to_excel('final_dataset.xlsx', index=False)


print(df.sort_values(by='Year', ascending=False).head())